In [2]:
!pip -q install pyarrow

import pandas as pd
import numpy as np
from pathlib import Path

# مسیرها
TRAIN = "/content/drive/MyDrive/train.csv"
TEST  = "/content/drive/MyDrive/test.csv"
OUT_DIR = Path("/content/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def read_csv_simple(path, chunksize=None):
    """
    خواندن بدون دردسر:
    - engine='python' برای انعطاف
    - encoding='utf-8' + encoding_errors='replace' تا هر بایت بد را جایگزین کند و کرش نکنیم
    - on_bad_lines='skip' برای رد کردن خطوط ناقص
    """
    return pd.read_csv(
        path,
        engine="python",
        encoding="utf-8",
        encoding_errors="replace",
        on_bad_lines="skip",
        chunksize=chunksize
    )

def normalize_columns(df):
    df = df.copy()
    df.columns = (df.columns
                  .str.strip()
                  .str.replace(r"\s+", "_", regex=True)
                  .str.lower())
    return df

def parse_common_datetimes(df):
    """
    فقط ستون‌های رایج را اگر وجود داشتند تبدیل می‌کنیم.
    نه چک dtype پیچیده، نه tz بازی؛ ساده و امن.
    """
    df = df.copy()
    dt_candidates = [
        "pickup_datetime", "dropoff_datetime",
        "datetime", "timestamp", "pickup_time", "dropoff_time",
        "date", "pickup_date", "dropoff_date"
    ]
    for c in dt_candidates:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)
    return df

def preprocess_minimal(df):
    df = normalize_columns(df)
    df = parse_common_datetimes(df)
    return df

def process_file(path, out_parquet, chunksize=None):
    if chunksize:
        # اگر رم کم باشد
        reader = read_csv_simple(path, chunksize=chunksize)
        part_paths = []
        for i, chunk in enumerate(reader):
            chunk = preprocess_minimal(chunk)
            p = OUT_DIR / f"{Path(out_parquet).stem}_part{i}.parquet"
            chunk.to_parquet(p, index=False)
            part_paths.append(p)
        print(f"Wrote {len(part_paths)} parts for {Path(path).name} → {OUT_DIR}")
        return part_paths
    else:
        df = read_csv_simple(path)
        df = preprocess_minimal(df)
        df.to_parquet(out_parquet, index=False)
        print(f"Saved {out_parquet} with shape {df.shape}")
        return [out_parquet]

# اگر خطای کمبود حافظه دیدی، این را True کن
USE_CHUNKS = False
CHUNK_SIZE = 200_000

train_out = OUT_DIR / "train_prepared.parquet"
test_out  = OUT_DIR / "test_prepared.parquet"

train_parts = process_file(TRAIN, train_out, chunksize=CHUNK_SIZE if USE_CHUNKS else None)
test_parts  = process_file(TEST,  test_out,  chunksize=CHUNK_SIZE if USE_CHUNKS else None)

# چک سریع
def peek(parquet_paths):
    import pandas as pd
    p = parquet_paths[0]
    df = pd.read_parquet(p)
    print("Path:", p)
    print("Shape:", df.shape)
    print("Columns:", list(df.columns)[:12], "...")
    print(df.head(3))

print("\n[TRAIN SAMPLE]")
peek(train_parts)
print("\n[TEST SAMPLE]")
peek(test_parts)


Saved /content/processed/train_prepared.parquet with shape (1281539, 11)
Saved /content/processed/test_prepared.parquet with shape (203550, 10)

[TRAIN SAMPLE]
Path: /content/processed/train_prepared.parquet
Shape: (1281539, 11)
Columns: ['id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'total_price'] ...
          id           pickup_datetime          dropoff_datetime  \
0  id3702621 2016-06-07 08:57:04+00:00 2016-06-07 09:05:47+00:00   
1  id2583306 2016-02-12 16:51:40+00:00 2016-02-12 16:56:23+00:00   
2  id0304728 2016-05-14 22:11:11+00:00 2016-05-14 22:14:49+00:00   

  passenger_count    pickup_longitude     pickup_latitude   dropoff_longitude  \
0               1  -73.95193481445312  40.777950286865234  -73.96041870117188   
1               2   -73.9742889404297   40.74249649047852  -73.97174072265625   
2               1  -73.97318267822266   40.

In [3]:
# نصب پیش‌نیازها (یک بار در ابتدای نوت‌بوک این سلول را اجرا کن)
!pip -q install --upgrade catboost pyarrow scikit-learn joblib

# چک نسخه برای اطمینان
import catboost, sklearn, pyarrow, joblib
print("catboost:", catboost.__version__)
print("sklearn:", sklearn.__version__)
print("pyarrow:", pyarrow.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 34.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
catboost: 1.2.8
sklearn: 1.6.1
pyarrow: 18.1.0


In [4]:


PROC = Path("/content/processed")
train_pq = PROC / "train_prepared.parquet"
test_pq  = PROC / "test_prepared.parquet"

# خواندن
train_df = pd.read_parquet(train_pq)
test_df  = pd.read_parquet(test_pq)

# حذف ستون‌های غیرمفید
cols_to_drop = ["id", "dropoff_datetime"]
train_df = train_df.drop(columns=[c for c in cols_to_drop if c in train_df.columns])
test_df  = test_df.drop(columns=[c for c in cols_to_drop if c in test_df.columns])

# ذخیرهٔ خروجی مرحله ۱
train_out = PROC / "train_step1_dropped.parquet"
test_out  = PROC / "test_step1_dropped.parquet"
train_df.to_parquet(train_out, index=False)
test_df.to_parquet(test_out, index=False)

print("✅ Done. Shapes:", train_df.shape, test_df.shape)


✅ Done. Shapes: (1281539, 9) (203550, 8)


In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

PROC = Path("/content/processed")
train_path = PROC / "train_step1_dropped.parquet"
test_path  = PROC / "test_step1_dropped.parquet"

train = pd.read_parquet(train_path)
test  = pd.read_parquet(test_path)

def audit_dataframe(df: pd.DataFrame, name: str):
    print(f"\n===== {name} =====")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))

    # --- 1) وضعیت مقادیر خالی (NaN)
    na_counts = df.isna().sum()
    na_pct = (na_counts / len(df) * 100).round(3)
    na_tbl = pd.DataFrame({"NaN_count": na_counts, "NaN_%": na_pct})
    na_tbl = na_tbl[na_tbl["NaN_count"] > 0].sort_values("NaN_count", ascending=False)
    print("\n[NaN summary — only columns with NaNs]")
    if len(na_tbl) == 0:
        print("No NaNs detected.")
    else:
        print(na_tbl.to_string())

    # --- 2) مدت سفر (فقط گزارش؛ بدون تغییر)
    if "trip_duration" in df.columns:
        dur = pd.to_numeric(df["trip_duration"], errors="coerce")
        dur_min = dur / 60.0

        n_missing = dur.isna().sum()
        n_nonpos  = (dur <= 0).sum()
        print("\n[Trip duration (seconds) — basic checks]")
        print(f"Missing: {n_missing} | Non-positive (<=0): {n_nonpos}")

        q = dur_min.quantile([0.25, 0.5, 0.75, 0.90, 0.95, 0.99]).round(2)
        print("\n[Trip duration (minutes) — quantiles]")
        print(q.to_string())
        print("Min (min):", round(dur_min.min(), 2), "| Max (min):", round(dur_min.max(), 2))

    # --- 3) Passenger count
    if "passenger_count" in df.columns:
        pc = pd.to_numeric(df["passenger_count"], errors="coerce")
        n_missing = pc.isna().sum()
        n_nonpos  = (pc <= 0).sum()
        n_ge_7    = (pc >= 7).sum()
        print("\n[Passenger count — basic checks]")
        print(f"Missing: {n_missing} | Non-positive (<=0): {n_nonpos} | >=7: {n_ge_7}")
        print("Value counts (top 10):")
        print(pc.value_counts(dropna=False).head(10).to_string())

    # --- 4) مختصات — فقط گزارش مین/مکس
    coord_cols = ["pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude"]
    if set(coord_cols).issubset(df.columns):
        print("\n[Coordinates — min/max]")
        stats = {}
        for c in coord_cols:
            s = pd.to_numeric(df[c], errors="coerce")
            stats[c] = {"min": float(np.nanmin(s)), "max": float(np.nanmax(s)), "NaNs": int(s.isna().sum())}
        print(pd.DataFrame(stats).T.to_string())

    # --- 5) زمان‌ها — پوشش تاریخی
    if "pickup_datetime" in df.columns:
        ts = pd.to_datetime(df["pickup_datetime"], errors="coerce", utc=True)
        print("\n[Pickup datetime — coverage]")
        print("Missing:", ts.isna().sum())
        if ts.notna().any():
            print("Min:", ts.min(), "| Max:", ts.max())

    # --- 6) store_and_fwd_flag
    if "store_and_fwd_flag" in df.columns:
        print("\n[store_and_fwd_flag — value counts]")
        print(df["store_and_fwd_flag"].astype(str).str.upper().value_counts(dropna=False).to_string())

    # --- نمونه چند ردیف برای نگاه سریع
    print("\n[Head — sample 3 rows]")
    print(df.head(3).to_string(index=False))


# اجرای آدیت روی هر دو مجموعه
audit_dataframe(train, "TRAIN (step1)")
audit_dataframe(test,  "TEST  (step1)")



===== TRAIN (step1) =====
Shape: (1281539, 9)
Columns: ['pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'total_price']

[NaN summary — only columns with NaNs]
                    NaN_count   NaN_%
pickup_datetime        212699  16.597
total_price            212698  16.597
trip_duration          212694  16.597
store_and_fwd_flag     212673  16.595
dropoff_latitude       212600  16.589
dropoff_longitude      212400  16.574
pickup_latitude        211826  16.529
pickup_longitude       210108  16.395
passenger_count        204970  15.994

[Trip duration (seconds) — basic checks]
Missing: 212699 | Non-positive (<=0): 0

[Trip duration (minutes) — quantiles]
0.25     6.62
0.50    11.03
0.75    17.93
0.90    27.25
0.95    35.07
0.99    57.50
Min (min): 0.02 | Max (min): 58771.37

[Passenger count — basic checks]
Missing: 212694 | Non-positive (<=0): 47 | >=7: 4
Value counts (top 10):
p

In [6]:
from pathlib import Path
import pandas as pd
import numpy as np

PROC = Path("/content/processed")
train_path = PROC / "train_step1_dropped.parquet"
test_path  = PROC / "test_step1_dropped.parquet"

train = pd.read_parquet(train_path)
test  = pd.read_parquet(test_path)

def audit_flag(df: pd.DataFrame, name: str, col: str = "store_and_fwd_flag"):
    print(f"\n===== {name} — {col} audit =====")
    s_raw = df[col]

    # نرمال‌سازی فقط برای گزارش (حروف بزرگ، حذف فاصله‌ها)
    s_norm = s_raw.astype(str).str.strip().str.upper()
    # تبدیل صریح 'NAN' رشته‌ای به NaN واقعی (در گزارش)
    s_norm = s_norm.replace({"NAN": np.nan})

    total = len(s_norm)
    n_nan = s_norm.isna().sum()

    # مقادیر مجاز
    valid = s_norm.isin(["Y","N"])
    n_valid = valid.sum()
    n_other = total - n_valid - n_nan

    print(f"Total rows: {total}")
    print(f"Valid (Y/N): {n_valid} ({n_valid/total:.2%})")
    print(f"Missing (NaN): {n_nan} ({n_nan/total:.2%})")
    print(f"Other (unexpected): {n_other} ({n_other/total:.2%})")

    if n_other > 0:
        print("\nTop unexpected tokens (normalized):")
        print(s_norm[~valid & s_norm.notna()].value_counts().head(10).to_string())

    # برای اطمینان، خام هم نشان بدهیم (بدون نرمال‌سازی) — فقط ۵ مورد اول
    if n_other > 0:
        print("\nSample of raw values for unexpected rows:")
        samp = s_raw[~valid & s_norm.notna()].head(5)
        for i, v in enumerate(samp, 1):
            print(f"{i}. {repr(v)}")

audit_flag(train, "TRAIN")
audit_flag(test,  "TEST")



===== TRAIN — store_and_fwd_flag audit =====
Total rows: 1281539
Valid (Y/N): 1068840 (83.40%)
Missing (NaN): 0 (0.00%)
Other (unexpected): 212699 (16.60%)

Top unexpected tokens (normalized):
store_and_fwd_flag
NONE                                                                                                                                                                                                                                                                                                                                                    212673
��~KTۮD�D�Y+S��{�����|���҆��R;�X'�R��!͚��!� �]ݢͶUBU�.�QE�<������7����D���Z�!��07H(�&+AG                                                                                                                                                                                                                                                         1
�\A��#��#�>����                                                                   

In [7]:
from pathlib import Path
import pandas as pd
import numpy as np

PROC = Path("/content/processed")
train_p = PROC / "train_step1_dropped.parquet"
test_p  = PROC / "test_step1_dropped.parquet"

train = pd.read_parquet(train_p)
test  = pd.read_parquet(test_p)

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1.astype(float))
    lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2.astype(float))
    lon2 = np.radians(lon2.astype(float))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2*R*np.arcsin(np.sqrt(a))

def apply_core_steps(df: pd.DataFrame, is_train: bool) -> tuple[pd.DataFrame, dict]:
    df = df.copy()
    rep = {}
    n0 = len(df)

    # 1) map store_and_fwd_flag → 0/1 (others → NaN)
    s = df["store_and_fwd_flag"].astype(str).str.strip().str.upper()
    df["store_and_fwd_flag_bin"] = s.map({"Y":1, "N":0})
    rep["flag_other_as_nan"] = (s.isin(["Y","N"]) == False).sum()

    # 2) numeric coercions (مهم: قبل از dropna)
    df["trip_duration"] = pd.to_numeric(df["trip_duration"], errors="coerce")
    df["trip_duration_min"] = df["trip_duration"] / 60.0

    # coords → numeric
    for c in ["pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude","passenger_count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

# 2.5) total_price → numeric (خیلی مهم)
    if is_train and "total_price" in df.columns:
        df["total_price"] = pd.to_numeric(df["total_price"], errors="coerce")
    # 3) drop NaNs on core columns
    core_cols = [
        "pickup_datetime","passenger_count",
        "pickup_longitude","pickup_latitude",
        "dropoff_longitude","dropoff_latitude",
        "trip_duration_min","store_and_fwd_flag_bin"
    ]
    if is_train and "total_price" in df.columns:
        core_cols.append("total_price")

    before = len(df)
    df = df.dropna(subset=[c for c in core_cols if c in df.columns])
    rep["drop_nan_core"] = before - len(df)

    # 4) passenger_count > 0
    before = len(df)
    df = df[df["passenger_count"] > 0]
    rep["drop_passenger_nonpos"] = before - len(df)

    # 5) distance (haversine, km) — الان همه عددی و غیر-NaN هستند
    df["distance_haversine_km"] = haversine_km(
        df["pickup_latitude"], df["pickup_longitude"],
        df["dropoff_latitude"], df["dropoff_longitude"]
    )

    # 6) duration cap: > 240 min (4h) → drop
    before = len(df)
    df = df[df["trip_duration_min"] <= 240]
    rep["drop_duration_gt_240m"] = before - len(df)

    # 7) impossible combo: distance == 0 & duration > 5 min → drop
    before = len(df)
    df = df[~((df["distance_haversine_km"] == 0) & (df["trip_duration_min"] > 5))]
    rep["drop_zero_dist_long_dur"] = before - len(df)

    rep["kept_rows"] = len(df)
    rep["dropped_total"] = n0 - len(df)
    return df, rep

train2, rep_tr = apply_core_steps(train, is_train=True)
test2,  rep_te = apply_core_steps(test,  is_train=False)

# ذخیره
out_train = PROC / "train_step2_clean.parquet"
out_test  = PROC / "test_step2_clean.parquet"
train2.to_parquet(out_train, index=False)
test2.to_parquet(out_test,  index=False)

# گزارش
print("=== REPORT — TRAIN ===")
for k,v in rep_tr.items(): print(f"{k:>24}: {v}")
print("shape:", train2.shape, "→", out_train)

print("\n=== REPORT — TEST ===")
for k,v in rep_te.items(): print(f"{k:>24}: {v}")
print("shape:", test2.shape, "→", out_test)


=== REPORT — TRAIN ===
       flag_other_as_nan: 212699
           drop_nan_core: 212700
   drop_passenger_nonpos: 46
   drop_duration_gt_240m: 1536
 drop_zero_dist_long_dur: 2340
               kept_rows: 1064917
           dropped_total: 216622
shape: (1064917, 12) → /content/processed/train_step2_clean.parquet

=== REPORT — TEST ===
       flag_other_as_nan: 0
           drop_nan_core: 0
   drop_passenger_nonpos: 6
   drop_duration_gt_240m: 267
 drop_zero_dist_long_dur: 466
               kept_rows: 202811
           dropped_total: 739
shape: (202811, 11) → /content/processed/test_step2_clean.parquet


In [8]:
from pathlib import Path
import pandas as pd
import numpy as np

PROC = Path("/content/processed")

def load_with_fallback(name_step2: str, name_step1: str):
    p2 = PROC / name_step2
    p1 = PROC / name_step1
    if p2.exists():
        df = pd.read_parquet(p2)
        src = str(p2)
    else:
        df = pd.read_parquet(p1)
        src = str(p1) + "  (fallback)"
    return df, src

def ensure_step2_features(df: pd.DataFrame, is_train: bool) -> pd.DataFrame:
    df = df.copy()

    # 1) store_and_fwd_flag_bin
    if "store_and_fwd_flag_bin" not in df.columns and "store_and_fwd_flag" in df.columns:
        s = df["store_and_fwd_flag"].astype(str).str.strip().str.upper()
        df["store_and_fwd_flag_bin"] = s.map({"Y":1, "N":0})

    # 2) trip_duration_min
    if "trip_duration_min" not in df.columns and "trip_duration" in df.columns:
        df["trip_duration"] = pd.to_numeric(df["trip_duration"], errors="coerce")
        df["trip_duration_min"] = df["trip_duration"] / 60.0

    # 3) distance_haversine_km
    need_dist = "distance_haversine_km" not in df.columns
    have_coords = {"pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude"}.issubset(df.columns)
    if need_dist and have_coords:
        for c in ["pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        R = 6371.0
        lat1 = np.radians(df["pickup_latitude"].astype(float))
        lon1 = np.radians(df["pickup_longitude"].astype(float))
        lat2 = np.radians(df["dropoff_latitude"].astype(float))
        lon2 = np.radians(df["dropoff_longitude"].astype(float))
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
        df["distance_haversine_km"] = 2*R*np.arcsin(np.sqrt(a))

    return df

def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "pickup_datetime" in df.columns and not pd.api.types.is_datetime64_any_dtype(df["pickup_datetime"]):
        df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], errors="coerce", utc=True)

    dt = df["pickup_datetime"]
    df["hour"]  = dt.dt.hour
    df["dow"]   = dt.dt.dayofweek
    df["month"] = dt.dt.month
    df["is_weekend"] = df["dow"].isin([5,6]).astype(int)

    df["sin_hour"] = np.sin(2*np.pi*df["hour"]/24.0)
    df["cos_hour"] = np.cos(2*np.pi*df["hour"]/24.0)
    df["sin_dow"]  = np.sin(2*np.pi*df["dow"]/7.0)
    df["cos_dow"]  = np.cos(2*np.pi*df["dow"]/7.0)

    # فقط دقیقه بماند
    if "trip_duration" in df.columns:
        df = df.drop(columns=["trip_duration"])

    return df

# --- Load (Step2 if exists, else Step1) ---
train, src_tr = load_with_fallback("train_step2_clean.parquet", "train_step1_dropped.parquet")
test,  src_te = load_with_fallback("test_step2_clean.parquet",  "test_step1_dropped.parquet")
print("[INPUT] TRAIN src:", src_tr, " | shape:", train.shape)
print("[INPUT] TEST  src:", src_te, " | shape:", test.shape)

# --- Ensure Step2 features present ---
train = ensure_step2_features(train, is_train=True)
test  = ensure_step2_features(test,  is_train=False)

# --- Add time features ---
train3 = add_time_features(train)
test3  = add_time_features(test)

# --- Save ---
out_train = PROC / "train_step3_time.parquet"
out_test  = PROC / "test_step3_time.parquet"
train3.to_parquet(out_train, index=False)
test3.to_parquet(out_test,  index=False)

# --- Quick check ---
need_cols_tr = ["pickup_datetime","trip_duration_min","distance_haversine_km","store_and_fwd_flag_bin","hour","dow","month","is_weekend"]
missing_tr = [c for c in need_cols_tr if c not in train3.columns]
need_cols_te = [c for c in need_cols_tr if c != "store_and_fwd_flag_bin"]  # test هم باید داشته باشد
missing_te = [c for c in need_cols_te if c not in test3.columns]

print("\n✅ Step3 fixed & saved.")
print("Train shape:", train3.shape, "→", out_train, "| missing:", missing_tr)
print("Test  shape:", test3.shape,  "→", out_test,  "| missing:", missing_te)

print("\n[TRAIN sample]")
print(train3.head(3)[[c for c in need_cols_tr if c in train3.columns]].to_string(index=False))
print("\n[TEST sample]")
print(test3.head(3)[[c for c in need_cols_te if c in test3.columns]].to_string(index=False))


[INPUT] TRAIN src: /content/processed/train_step2_clean.parquet  | shape: (1064917, 12)
[INPUT] TEST  src: /content/processed/test_step2_clean.parquet  | shape: (202811, 11)

✅ Step3 fixed & saved.
Train shape: (1064917, 19) → /content/processed/train_step3_time.parquet | missing: []
Test  shape: (202811, 18) → /content/processed/test_step3_time.parquet | missing: []

[TRAIN sample]
          pickup_datetime  trip_duration_min  distance_haversine_km  store_and_fwd_flag_bin  hour  dow  month  is_weekend
2016-06-07 08:57:04+00:00           8.716667               1.484946                     0.0     8    1      6           0
2016-02-12 16:51:40+00:00           4.716667               0.823789                     0.0    16    4      2           0
2016-05-14 22:11:11+00:00           3.633333               0.630291                     0.0    22    5      5           1

[TEST sample]
          pickup_datetime  trip_duration_min  distance_haversine_km  hour  dow  month  is_weekend
2016-04-06 19

In [9]:
# Step5 — Robust 80/20 time-based split (self-contained & resilient)
import json, numpy as np, pandas as pd
from pathlib import Path
import numpy as np

PROC = Path("/content/processed")
ART  = PROC / "artifacts"
ART.mkdir(parents=True, exist_ok=True)

X_path     = PROC / "X_train.parquet"
y_path     = PROC / "y_train.parquet"
Xtest_path = PROC / "X_test.parquet"

# --- helpers (local, self-contained) ---
def _ensure_numeric(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def _add_time_features_inplace(df):
    if "pickup_datetime" in df.columns and not pd.api.types.is_datetime64_any_dtype(df["pickup_datetime"]):
        df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], errors="coerce", utc=True)
    dt = df["pickup_datetime"]
    df["hour"]  = dt.dt.hour
    df["dow"]   = dt.dt.dayofweek
    df["month"] = dt.dt.month
    df["is_weekend"] = df["dow"].isin([5,6]).astype(int)
    df["sin_hour"] = np.sin(2*np.pi*df["hour"]/24.0)
    df["cos_hour"] = np.cos(2*np.pi*df["hour"]/24.0)
    df["sin_dow"]  = np.sin(2*np.pi*df["dow"]/7.0)
    df["cos_dow"]  = np.cos(2*np.pi*df["dow"]/7.0)

def _manhattan_km(lat1, lon1, lat2, lon2):
    lat_km  = 110.574 * np.abs(lat2 - lat1)
    mean_lat = np.radians((lat1 + lat2) / 2.0)
    lon_km  = (111.320 * np.cos(mean_lat)) * np.abs(lon2 - lon1)
    return lat_km + lon_km

def _bearing_deg(lat1, lon1, lat2, lon2):
    lat1r = np.radians(lat1); lat2r = np.radians(lat2)
    dlon = np.radians(lon2 - lon1)
    y = np.sin(dlon) * np.cos(lat2r)
    x = np.cos(lat1r)*np.cos(lat2r) - np.sin(lat1r)*np.sin(lat2r)*np.cos(dlon)
    br = np.degrees(np.arctan2(y, x))
    return (br + 360.0) % 360.0

def _haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1.astype(float))
    lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2.astype(float))
    lon2 = np.radians(lon2.astype(float))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2*R*np.arcsin(np.sqrt(a))

def _add_geo_flags_inplace(df):
    # نیازهای عددی
    need = ["pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude",
            "passenger_count","trip_duration_min"]
    _ensure_numeric(df, [c for c in need if c in df.columns])

    # distance_haversine اگر نیست
    if "distance_haversine_km" not in df.columns:
        df["distance_haversine_km"] = _haversine_km(
            df["pickup_latitude"], df["pickup_longitude"],
            df["dropoff_latitude"], df["dropoff_longitude"]
        )

    # Manhattan + bearing
    df["distance_manhattan_km"] = _manhattan_km(
        df["pickup_latitude"], df["pickup_longitude"],
        df["dropoff_latitude"], df["dropoff_longitude"]
    )
    df["bearing_deg"] = _bearing_deg(
        df["pickup_latitude"], df["pickup_longitude"],
        df["dropoff_latitude"], df["dropoff_longitude"]
    )

    # flags
    df["is_short_trip"] = (df["distance_haversine_km"] <= 2.0).astype(int)
    df["is_long_trip"]  = (df["distance_haversine_km"] >= 20.0).astype(int)
    df["passenger_ge3"] = (df["passenger_count"] >= 3).astype(int)
    df["passenger_ge5"] = (df["passenger_count"] >= 5).astype(int)

    # rush
    if "hour" not in df.columns and "pickup_datetime" in df.columns:
        df["hour"] = pd.to_datetime(df["pickup_datetime"], errors="coerce", utc=True).dt.hour
    df["is_rush"] = df["hour"].between(7,9).astype(int) | df["hour"].between(16,18).astype(int)

def rebuild_from_any_available_step():
    """
    اگر Step4 (X/y) نبود: تلاش می‌کنیم از Step3 بسازیم؛
    اگر Step3 هم نبود: از Step2 یا Step1 بازسازی کامل انجام می‌دهیم.
    """
    # 1) اگر X/y already exist → تمام
    if X_path.exists() and y_path.exists() and Xtest_path.exists():
        print("✅ X/y already exist. No rebuild needed.")
        return

    # 2) سعی کن Step3 را بخوانی
    tr3 = PROC / "train_step3_time.parquet"
    te3 = PROC / "test_step3_time.parquet"
    if tr3.exists() and te3.exists():
        train = pd.read_parquet(tr3)
        test  = pd.read_parquet(te3)
        print("ℹ️ Using Step3 files.")
    else:
        # 3) اگر Step3 نبود، از Step2 یا Step1 بخوان
        candidate_pairs = [
            ("train_step2_clean.parquet", "test_step2_clean.parquet"),
            ("train_step1_dropped.parquet","test_step1_dropped.parquet"),
        ]
        train = test = None
        for a,b in candidate_pairs:
            pa, pb = PROC / a, PROC / b
            if pa.exists() and pb.exists():
                train = pd.read_parquet(pa)
                test  = pd.read_parquet(pb)
                print(f"ℹ️ Building Step3 from {a}/{b}.")
                break
        if train is None:
            raise FileNotFoundError(
                "هیچ‌کدام از خروجی‌های Step3/Step2/Step1 یافت نشد. اول سلول‌های قبل را اجرا کن."
            )

        # ساخت Step3 روی پرواز
        # اطمینان از ستون‌های لازم
        train = _ensure_numeric(train, ["trip_duration"])
        test  = _ensure_numeric(test,  ["trip_duration"])
        if "trip_duration_min" not in train.columns and "trip_duration" in train.columns:
            train["trip_duration_min"] = pd.to_numeric(train["trip_duration"], errors="coerce") / 60.0
        if "trip_duration_min" not in test.columns and "trip_duration" in test.columns:
            test["trip_duration_min"] = pd.to_numeric(test["trip_duration"], errors="coerce") / 60.0

        _add_time_features_inplace(train)
        _add_time_features_inplace(test)

        # ذخیره‌ی Step3 برای دفعات بعد
        train.to_parquet(tr3, index=False)
        test.to_parquet(te3,  index=False)
        print("✅ Step3 regenerated and saved.")

    # 4) اضافه‌کردن Geo & flags (Step4-light)
    _add_geo_flags_inplace(train)
    _add_geo_flags_inplace(test)

    # 5) جداسازی هدف و ذخیره X/y
      # 5) جداسازی هدف و ذخیره X/y (با تمیزکاری و هم‌ترازی)
    if "total_price" not in train.columns:
        raise ValueError("total_price در train موجود نیست. مطمئن شو در دیتای train هست.")

    # y را عددی و امن کن
    y_raw = train["total_price"]
    y_num = pd.to_numeric(y_raw, errors="coerce")

    # ماسک: فقط مقادیر عددی، متناهی، و غیرمنفی
    mask = y_num.notna() & np.isfinite(y_num) & (y_num >= 0)

    dropped = int((~mask).sum())
    if dropped:
        print(f"⚠️ dropping {dropped} rows due to non-numeric/negative targets (train).")

    # هم‌ترازی X و y با همان ماسک
    Xtr = train.loc[mask].drop(columns=["total_price"]).reset_index(drop=True)
    y   = y_num.loc[mask].reset_index(drop=True)

    # برای test مطمئن شو ستون total_price ندارد
    Xte = test.copy()
    if "total_price" in Xte.columns:
        Xte = Xte.drop(columns=["total_price"])

    # ذخیره
    Xtr.to_parquet(X_path, index=False)
    Xte.to_parquet(Xtest_path, index=False)
    pd.DataFrame({"total_price": y}).to_parquet(y_path, index=False)
    print("🔁 Rebuilt X_train/X_test/y_train and saved (clean).")

# === اجرا ===
rebuild_from_any_available_step()

# --- حالا اسپلیت زمانی 80/20 ---
X_full = pd.read_parquet(X_path)
y_full = pd.read_parquet(y_path)["total_price"].values

assert "pickup_datetime" in X_full.columns, "pickup_datetime باید داخل X_train باشد برای اسپلیت زمانی."
ts = pd.to_datetime(X_full["pickup_datetime"], utc=True, errors="coerce")
assert ts.notna().all(), "pickup_datetime شامل NaT است—باید سالم باشد."

cutoff = ts.quantile(0.80)  # آخرین 20٪ بازه زمانی → ولیدیشن
train_mask = ts < cutoff
valid_mask = ~train_mask

X_tr, X_va = X_full.loc[train_mask].copy(), X_full.loc[valid_mask].copy()
y_tr, y_va = y_full[train_mask.values], y_full[valid_mask.values]

# datetime را از فیچرها حذف می‌کنیم
for df_ in (X_tr, X_va):
    df_.drop(columns=["pickup_datetime"], inplace=True, errors="ignore")

# ذخیره
X_tr_path = PROC / "X_tr.parquet"
X_va_path = PROC / "X_va.parquet"
y_tr_path = PROC / "y_tr.parquet"
y_va_path = PROC / "y_va.parquet"
info_path  = ART / "time_split_info.json"

X_tr.to_parquet(X_tr_path, index=False)
X_va.to_parquet(X_va_path, index=False)
pd.DataFrame({"total_price": y_tr}).to_parquet(y_tr_path, index=False)
pd.DataFrame({"total_price": y_va}).to_parquet(y_va_path, index=False)
with open(info_path, "w") as f:
    json.dump({"cutoff_iso": cutoff.isoformat(),
               "n_train": int(train_mask.sum()),
               "n_valid": int(valid_mask.sum())}, f, indent=2)

print("✅ Time-based split done.")
print("Cutoff:", cutoff)
print("Train:", X_tr.shape, "| Valid:", X_va.shape)
print("Saved →", X_tr_path, X_va_path, y_tr_path, y_va_path, info_path)


ℹ️ Using Step3 files.
🔁 Rebuilt X_train/X_test/y_train and saved (clean).
✅ Time-based split done.
Cutoff: 2016-05-25 06:01:21.800000+00:00
Train: (851933, 24) | Valid: (212984, 24)
Saved → /content/processed/X_tr.parquet /content/processed/X_va.parquet /content/processed/y_tr.parquet /content/processed/y_va.parquet /content/processed/artifacts/time_split_info.json


In [10]:
# ===========================================
# HonkHonk — Modeling (ElasticNet + CatBoost)
# ===========================================
# ورودی‌ها (از Step5):
#   /content/processed/X_tr.parquet, X_va.parquet, y_tr.parquet, y_va.parquet
# خروجی‌ها:
#   /content/processed/artifacts/*
# -------------------------------------------

import os, json, numpy as np, pandas as pd
from pathlib import Path

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

from catboost import CatBoostRegressor, Pool
from catboost.core import CatBoostError

PROC = Path("/content/processed")
ART  = PROC / "artifacts"
ART.mkdir(parents=True, exist_ok=True)

# ---------- Load data ----------
X_tr = pd.read_parquet(PROC / "X_tr.parquet")
X_va = pd.read_parquet(PROC / "X_va.parquet")
y_tr = pd.read_parquet(PROC / "y_tr.parquet")["total_price"].values
y_va = pd.read_parquet(PROC / "y_va.parquet")["total_price"].values

print("Loaded:", X_tr.shape, X_va.shape, y_tr.shape, y_va.shape)

# ---------- Coerce numerics in X ----------
for df in (X_tr, X_va):
    for c in df.columns:
        if not np.issubdtype(df[c].dtype, np.number):
            df[c] = pd.to_numeric(df[c], errors="coerce")

# Replace ±inf → NaN for imputation
for df in (X_tr, X_va):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop columns that are all-NaN in TRAIN (and mirror on VALID)
def drop_all_nan_cols(df_tr: pd.DataFrame, df_va: pd.DataFrame):
    keep_mask = df_tr.notna().any(axis=0)
    keep_cols = df_tr.columns[keep_mask]
    dropped = [c for c in df_tr.columns if c not in keep_cols]
    if dropped:
        print(f"⚠️ Dropping all-NaN train columns (mirrored on valid): {dropped[:10]}{'...' if len(dropped)>10 else ''}")
    return df_tr[keep_cols].copy(), df_va[keep_cols].copy()

X_tr, X_va = drop_all_nan_cols(X_tr, X_va)

# ---------- Metrics helper ----------
def eval_on_valid(y_true, y_pred, label):
    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    r2   = float(r2_score(y_true, y_pred))
    print(f"[{label}] MAE={mae:,.4f} | RMSE={rmse:,.4f} | R2={r2:,.4f}")
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

# ---------- Sanitize targets (numeric, non-negative, aligned with X) ----------
def fix_target(X: pd.DataFrame, y_array, split_name: str):
    y = pd.Series(y_array, name="total_price")
    y_num = pd.to_numeric(y, errors="coerce")
    mask = y_num.notna() & np.isfinite(y_num) & (y_num >= 0)
    dropped = int((~mask).sum())
    if dropped:
        print(f"⚠️ {split_name}: dropping {dropped} rows due to non-numeric/negative targets.")
        X = X.loc[mask.values].reset_index(drop=True)
        y_num = y_num.loc[mask].reset_index(drop=True)
    return X, y_num.to_numpy(dtype=float)

X_tr, y_tr = fix_target(X_tr, y_tr, "train")
X_va, y_va = fix_target(X_va, y_va, "valid")

# ذخیرهٔ نسخهٔ تمیز و هم‌تراز
X_tr.to_parquet(PROC / "X_tr.parquet", index=False)
X_va.to_parquet(PROC / "X_va.parquet", index=False)
pd.DataFrame({"total_price": y_tr}).to_parquet(PROC / "y_tr.parquet", index=False)
pd.DataFrame({"total_price": y_va}).to_parquet(PROC / "y_va.parquet", index=False)

print(f"Targets fixed. Shapes → X_tr: {X_tr.shape}  y_tr: {y_tr.shape}  |  X_va: {X_va.shape}  y_va: {y_va.shape}")

# =========================================================
# 1) Baseline Linear (ElasticNet) — log-target + Imputer + RobustScaler
# =========================================================
print("\n=== 1) ElasticNet (log-target) ===")

y_tr_log = np.log1p(y_tr)

# برای کنترل زمان CV روی کپ اولیه
cv_max = 300_000
X_cv = X_tr.iloc[:min(len(X_tr), cv_max)].copy()
y_cv = y_tr_log[:min(len(y_tr_log), cv_max)].copy()

tscv = TimeSeriesSplit(n_splits=5)

pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  RobustScaler(with_centering=True, with_scaling=True)),
    ("enet",    ElasticNet(max_iter=2000, fit_intercept=True))
])

param_grid = {
    "enet__alpha":    [0.01, 0.03, 0.1, 0.3, 1.0],
    "enet__l1_ratio": [0.1, 0.5, 0.9],
}

gscv = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=tscv,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
    verbose=1
)
gscv.fit(X_cv, y_cv)
print("Best EN params:", gscv.best_params_, " | best CV MAE(log-space) =", -gscv.best_score_)

best_enet = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  RobustScaler(with_centering=True, with_scaling=True)),
    ("enet",    ElasticNet(
                   alpha=gscv.best_params_["enet__alpha"],
                   l1_ratio=gscv.best_params_["enet__l1_ratio"],
                   max_iter=4000,
                   fit_intercept=True))
])
best_enet.fit(X_tr, y_tr_log)

y_hat_en_log = best_enet.predict(X_va)
y_hat_en = np.expm1(y_hat_en_log)
metrics_en = eval_on_valid(y_va, y_hat_en, "ElasticNet-logtarget")

enet_path = ART / "elasticnet_logtarget.joblib"
joblib.dump(best_enet, enet_path)
print("Saved:", enet_path)

# =========================================================
# 2) CatBoost (GPU→CPU fallback) — Tuning (plain & log)
#    Grid (light): depth ∈ {6,8}, l2 ∈ {4,6,10}, lr ∈ {0.03,0.05,0.07}
#    Selection metric: MAE on VALID in original scale
# =========================================================
from itertools import product

def fit_catboost_once(X_tr, y_tr_use, X_va, y_va_use, base_params):
    train_pool = Pool(X_tr, y_tr_use)
    valid_pool = Pool(X_va, y_va_use)
    model = CatBoostRegressor(**base_params)
    try:
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)
    except CatBoostError as e:
        # GPU → CPU fallback
        base_params_cpu = base_params.copy()
        base_params_cpu["task_type"] = "CPU"
        base_params_cpu.pop("devices", None)
        print("⚠️ GPU failed; falling back to CPU. Reason:", str(e)[:180], "...")
        model = CatBoostRegressor(**base_params_cpu)
        model.fit(train_pool, eval_set=valid_pool, use_best_model=True)
    return model

def tune_catboost_variant(variant: str):
    """
    variant ∈ {"plain","log"}:
      - plain: روی y اصلی
      - log  : روی log1p(y) آموزش، ارزیابی روی مقیاس اصلی با expm1(pred)
    """
    print(f"\n=== Tuning CatBoost — {variant} ===")
    y_tr_use = y_tr if variant == "plain" else np.log1p(y_tr)
    y_va_use = y_va if variant == "plain" else np.log1p(y_va)

    base = {
        "loss_function": "RMSE",
        "eval_metric":   "MAE",
        "iterations":    5000,
        "learning_rate": 0.05,   # overridden by grid
        "depth":         8,      # overridden by grid
        "l2_leaf_reg":   6.0,    # overridden by grid
        "bootstrap_type":"Bernoulli",
        "subsample":     0.8,
        "random_strength": 1.0,
        "od_type":       "Iter",
        "od_wait":       200,
        "random_seed":   42,
        "allow_writing_files": False,
        "task_type": "GPU",
        "devices": "0",
        "verbose": 200,
    }

    grid = {
        "depth": [6, 8],
        "l2_leaf_reg": [4.0, 6.0, 10.0],
        "learning_rate": [0.03, 0.05, 0.07],
    }

    results = []
    best = {"mae": np.inf, "model": None, "params": None, "pred": None, "best_iter": None}

    for d, l2, lr in product(grid["depth"], grid["l2_leaf_reg"], grid["learning_rate"]):
        params = base.copy()
        params.update({"depth": d, "l2_leaf_reg": l2, "learning_rate": lr})
        print(f"\n--- Try: depth={d}, l2={l2}, lr={lr} ---")
        model = fit_catboost_once(X_tr, y_tr_use, X_va, y_va_use, params)
        pred = model.predict(X_va)
        pred_eval = np.expm1(pred) if variant == "log" else pred

        # ارزیابی روی مقیاس اصلی
        cur = eval_on_valid(y_va, pred_eval, f"CatBoost-{variant}-d{d}-l2{l2}-lr{lr}")
        mae, rmse, r2 = cur["MAE"], cur["RMSE"], cur["R2"]

        results.append({
            "variant": variant, "depth": d, "l2": l2, "lr": lr,
            "MAE": mae, "RMSE": rmse, "R2": r2,
            "best_iter": int(model.get_best_iteration())
        })

        if mae < best["mae"]:
            best.update({
                "mae": mae, "model": model, "params": {"depth": d, "l2_leaf_reg": l2, "learning_rate": lr},
                "pred": pred_eval, "best_iter": int(model.get_best_iteration())
            })

    # ذخیرهٔ نتایج تیونینگ و بهترین مدل
    df_res = pd.DataFrame(results).sort_values("MAE")
    res_path = ART / f"catboost_tuning_results_{variant}.csv"
    df_res.to_csv(res_path, index=False)
    print(f"\nBest {variant}: params={best['params']} | best_iter={best['best_iter']} | MAE={best['mae']:.4f}")
    print("Saved grid results →", res_path)

    # ذخیره مدل و فیچرایمپورنس برای بهترین
    best_path = ART / f"catboost_{variant}_tuned.cbm"
    best["model"].save_model(best_path)
    fi = best["model"].get_feature_importance(type="PredictionValuesChange")
    fi_tbl = pd.DataFrame({"feature": X_tr.columns, "importance": fi}).sort_values("importance", ascending=False)
    fi_path = ART / f"catboost_{variant}_tuned_feature_importance.csv"
    fi_tbl.to_csv(fi_path, index=False)
    print("Saved best model:", best_path, "| FI →", fi_path)

    return best, df_res

# اجرای تیونینگ برای هر دو و ساخت خلاصه
best_plain, res_plain = tune_catboost_variant("plain")
best_log,   res_log   = tune_catboost_variant("log")

summary = {
    "ElasticNet_logtarget": metrics_en,
    "CatBoost_plain_tuned": {
        "MAE": float(best_plain["mae"]),
        "RMSE": float(res_plain.sort_values("MAE").iloc[0]["RMSE"]),
        "R2": float(res_plain.sort_values("MAE").iloc[0]["R2"]),
        "best_params": best_plain["params"],
        "best_iter": int(best_plain["best_iter"])
    },
    "CatBoost_log_tuned": {
        "MAE": float(best_log["mae"]),
        "RMSE": float(res_log.sort_values("MAE").iloc[0]["RMSE"]),
        "R2": float(res_log.sort_values("MAE").iloc[0]["R2"]),
        "best_params": best_log["params"],
        "best_iter": int(best_log["best_iter"])
    }
}

with open(ART / "model_metrics_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n=== Summary (tuned) ===")
print(json.dumps(summary, indent=2))
print("Saved →", ART / "model_metrics_summary.json")


Loaded: (851933, 24) (212984, 24) (851933,) (212984,)
⚠️ Dropping all-NaN train columns (mirrored on valid): ['store_and_fwd_flag']
Targets fixed. Shapes → X_tr: (851933, 23)  y_tr: (851933,)  |  X_va: (212984, 23)  y_va: (212984,)

=== 1) ElasticNet (log-target) ===
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best EN params: {'enet__alpha': 0.01, 'enet__l1_ratio': 0.1}  | best CV MAE(log-space) = 0.1278290995957329
[ElasticNet-logtarget] MAE=3.6882 | RMSE=59.5958 | R2=-12.9659
Saved: /content/processed/artifacts/elasticnet_logtarget.joblib

=== Tuning CatBoost — plain ===

--- Try: depth=6, l2=4.0, lr=0.03 ---


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 9.7415125	test: 10.0284587	best: 10.0284587 (0)	total: 68.4ms	remaining: 5m 41s
200:	learn: 1.8030475	test: 1.6938925	best: 1.6938925 (200)	total: 2.44s	remaining: 58.3s
400:	learn: 1.6054734	test: 1.4807183	best: 1.4807183 (400)	total: 4.08s	remaining: 46.8s
600:	learn: 1.5345638	test: 1.4250225	best: 1.4250225 (600)	total: 5.69s	remaining: 41.6s
800:	learn: 1.4944379	test: 1.3995184	best: 1.3995184 (800)	total: 7.29s	remaining: 38.2s
1000:	learn: 1.4658333	test: 1.3820487	best: 1.3820487 (1000)	total: 8.83s	remaining: 35.3s
1200:	learn: 1.4472803	test: 1.3744919	best: 1.3744207 (1185)	total: 10.4s	remaining: 33s
1400:	learn: 1.4334598	test: 1.3674720	best: 1.3674720 (1400)	total: 13.8s	remaining: 35.6s
1600:	learn: 1.4213826	test: 1.3637440	best: 1.3630282 (1548)	total: 15.9s	remaining: 33.9s
1800:	learn: 1.4112200	test: 1.3605616	best: 1.3603620 (1784)	total: 17.5s	remaining: 31.1s
2000:	learn: 1.4026784	test: 1.3591059	best: 1.3586942 (1962)	total: 19.1s	remaining: 28.6s


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 9.5750628	test: 9.8553600	best: 9.8553600 (0)	total: 10.8ms	remaining: 54.2s
200:	learn: 1.6498900	test: 1.5192630	best: 1.5192630 (200)	total: 5.26s	remaining: 2m 5s
400:	learn: 1.5187462	test: 1.4129010	best: 1.4129010 (400)	total: 6.92s	remaining: 1m 19s
600:	learn: 1.4683305	test: 1.3908295	best: 1.3902340 (597)	total: 8.52s	remaining: 1m 2s
800:	learn: 1.4399517	test: 1.3794105	best: 1.3794105 (800)	total: 10.1s	remaining: 53s
1000:	learn: 1.4199422	test: 1.3731576	best: 1.3730886 (995)	total: 11.7s	remaining: 46.8s
1200:	learn: 1.4071347	test: 1.3725457	best: 1.3710004 (1104)	total: 13.3s	remaining: 42s
bestTest = 1.371000433
bestIteration = 1104
Shrink model to first 1105 iterations.
[CatBoost-plain-d6-l24.0-lr0.05] MAE=1.3710 | RMSE=2.7228 | R2=0.9708

--- Try: depth=6, l2=4.0, lr=0.07 ---


...

0:	learn: 0.3998328	test: 0.4086051	best: 0.4086051 (0)	total: 11.7ms	remaining: 58.5s
200:	learn: 0.0682262	test: 0.0615210	best: 0.0615210 (200)	total: 2.19s	remaining: 52.2s
400:	learn: 0.0591728	test: 0.0533928	best: 0.0533928 (400)	total: 4.37s	remaining: 50.1s
600:	learn: 0.0563303	test: 0.0519825	best: 0.0519642 (587)	total: 8.48s	remaining: 1m 2s
800:	learn: 0.0547872	test: 0.0513908	best: 0.0513815 (791)	total: 10.6s	remaining: 55.7s
1000:	learn: 0.0538003	test: 0.0510239	best: 0.0510217 (999)	total: 12.8s	remaining: 51.1s
1200:	learn: 0.0530385	test: 0.0508034	best: 0.0507656 (1168)	total: 14.9s	remaining: 47.2s
1400:	learn: 0.0525282	test: 0.0507051	best: 0.0506837 (1383)	total: 17s	remaining: 43.8s
1600:	learn: 0.0521126	test: 0.0507140	best: 0.0506654 (1576)	total: 20.5s	remaining: 43.6s
bestTest = 0.050665443
bestIteration = 1576
Shrink model to first 1577 iterations.
[CatBoost-log-d8-l210.0-lr0.03] MAE=1.2802 | RMSE=2.7246 | R2=0.9708

--- Try: depth=8, l2=10.0, lr=0.05 

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.3926675	test: 0.4013392	best: 0.4013392 (0)	total: 15.5ms	remaining: 1m 17s
200:	learn: 0.0611066	test: 0.0550943	best: 0.0550943 (200)	total: 2.18s	remaining: 52.1s
400:	learn: 0.0555386	test: 0.0514838	best: 0.0514838 (400)	total: 4.34s	remaining: 49.7s
600:	learn: 0.0536891	test: 0.0510425	best: 0.0510354 (597)	total: 6.47s	remaining: 47.4s
800:	learn: 0.0527478	test: 0.0509179	best: 0.0507392 (735)	total: 10.6s	remaining: 55.6s
bestTest = 0.05073924549
bestIteration = 735
Shrink model to first 736 iterations.
[CatBoost-log-d8-l210.0-lr0.05] MAE=1.2807 | RMSE=2.7159 | R2=0.9710

--- Try: depth=8, l2=10.0, lr=0.07 ---


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.3855276	test: 0.3940991	best: 0.3940991 (0)	total: 11.8ms	remaining: 59.1s
200:	learn: 0.0578938	test: 0.0525630	best: 0.0525630 (200)	total: 2.22s	remaining: 53s
400:	learn: 0.0537577	test: 0.0512096	best: 0.0511416 (394)	total: 4.37s	remaining: 50.1s
600:	learn: 0.0525153	test: 0.0508308	best: 0.0508221 (595)	total: 6.5s	remaining: 47.6s
bestTest = 0.05082207154
bestIteration = 595
Shrink model to first 596 iterations.
[CatBoost-log-d8-l210.0-lr0.07] MAE=1.2824 | RMSE=2.7245 | R2=0.9708

Best log: params={'depth': 8, 'l2_leaf_reg': 6.0, 'learning_rate': 0.03} | best_iter=1053 | MAE=1.2747
Saved grid results → /content/processed/artifacts/catboost_tuning_results_log.csv
Saved best model: /content/processed/artifacts/catboost_log_tuned.cbm | FI → /content/processed/artifacts/catboost_log_tuned_feature_importance.csv

=== Summary (tuned) ===
{
  "ElasticNet_logtarget": {
    "MAE": 3.688177072188429,
    "RMSE": 59.59578093605666,
    "R2": -12.9658778527889
  },
  "CatBoost

In [11]:
# === Eval R2/MAE/RMSE on VALID (holdout) for saved models ===
!pip -q install catboost pyarrow joblib

import os, json, numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import joblib

PROC = Path("/content/processed")
ART  = PROC / "artifacts"

# ---- Load VALID split
X_va = pd.read_parquet(PROC / "X_va.parquet")
y_va = pd.read_parquet(PROC / "y_va.parquet")["total_price"].values

# enforce numeric for safety (rare dtypes)
for c in X_va.columns:
    if not np.issubdtype(X_va[c].dtype, np.number):
        X_va[c] = pd.to_numeric(X_va[c], errors="coerce")

# drop any rows with non-numeric/negative targets (like your train step)
y_va_num = pd.to_numeric(pd.Series(y_va), errors="coerce")
mask = y_va_num.notna() & np.isfinite(y_va_num) & (y_va_num >= 0)
if (~mask).sum():
    print(f"⚠️ valid: dropping {(~mask).sum()} rows due to non-numeric/negative targets.")
X_va = X_va.loc[mask.values].reset_index(drop=True)
y_va = y_va_num.loc[mask].to_numpy(dtype=float)

def report(y_true, y_pred, label):
    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    r2   = float(r2_score(y_true, y_pred))
    print(f"[{label}]  MAE={mae:,.4f} | RMSE={rmse:,.4f} | R2={r2:,.6f}")
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

results = {}

# ---- 1) ElasticNet (log-target) pipeline
enet_path = ART / "elasticnet_logtarget.joblib"
if enet_path.exists():
    enet = joblib.load(enet_path)
    y_hat_log = enet.predict(X_va)          # model trained on log1p(y)
    y_hat     = np.expm1(y_hat_log)         # back to original scale
    results["ElasticNet_logtarget"] = report(y_va, y_hat, "ElasticNet-logtarget")
else:
    print("ℹ️ elasticnet_logtarget.joblib not found; skipping.")

# ---- 2) CatBoost—plain tuned
cb_plain_path = ART / "catboost_plain_tuned.cbm"
if cb_plain_path.exists():
    cb_plain = CatBoostRegressor()
    cb_plain.load_model(cb_plain_path)
    y_hat_plain = cb_plain.predict(X_va)
    results["CatBoost_plain_tuned"] = report(y_va, y_hat_plain, "CatBoost-plain (tuned)")
else:
    print("ℹ️ catboost_plain_tuned.cbm not found; skipping.")

# ---- 3) CatBoost—log tuned (needs expm1 on predictions)
cb_log_path = ART / "catboost_log_tuned.cbm"
if cb_log_path.exists():
    cb_log = CatBoostRegressor()
    cb_log.load_model(cb_log_path)
    y_hat_log = cb_log.predict(X_va)
    y_hat     = np.expm1(y_hat_log)
    results["CatBoost_log_tuned"] = report(y_va, y_hat, "CatBoost-log (tuned)")
else:
    print("ℹ️ catboost_log_tuned.cbm not found; skipping.")

print("\n=== Summary on VALID ===")
print(json.dumps(results, indent=2))


[ElasticNet-logtarget]  MAE=3.6882 | RMSE=59.5958 | R2=-12.965878
[CatBoost-plain (tuned)]  MAE=1.3440 | RMSE=2.7259 | R2=0.970782
[CatBoost-log (tuned)]  MAE=1.2747 | RMSE=2.7156 | R2=0.971002

=== Summary on VALID ===
{
  "ElasticNet_logtarget": {
    "MAE": 3.688177072188429,
    "RMSE": 59.59578093605666,
    "R2": -12.9658778527889
  },
  "CatBoost_plain_tuned": {
    "MAE": 1.3439523977620846,
    "RMSE": 2.7258696212901383,
    "R2": 0.9707822096565165
  },
  "CatBoost_log_tuned": {
    "MAE": 1.2746886792981627,
    "RMSE": 2.7155846372263657,
    "R2": 0.9710022771084897
  }
}


In [12]:
# === Evaluate on the EXACT saved VALID split ===
!pip -q install catboost pyarrow joblib

import json, numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
import joblib

PROC = Path("/content/processed")
ART  = PROC / "artifacts"

# 1) Load saved VALID split (the exact one used in training)
X_va = pd.read_parquet(PROC / "X_va.parquet")
# 1) Load saved VALID split (the exact one used in training)
X_va = pd.read_parquet(PROC / "X_va.parquet")

y_raw = pd.read_parquet(PROC / "y_va.parquet")["total_price"].astype(str)
y_clean = (y_raw.str.replace(",", "", regex=False).str.strip()
                  .str.replace(r"[^0-9eE\+\-\.]", "", regex=True))
y_va_num = pd.to_numeric(y_clean, errors="coerce")
mask = y_va_num.notna() & np.isfinite(y_va_num)
if (~mask).sum():
    print(f"⚠️ Valid split: dropping {(~mask).sum()} bad target rows.")
X_va = X_va.loc[mask.values].reset_index(drop=True)
y_va = y_va_num.loc[mask].to_numpy(dtype=float)


print("VALID shapes:", X_va.shape, y_va.shape)

def report(y_true, y_pred, label):
    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    r2   = float(r2_score(y_true, y_pred))
    print(f"[{label}]  MAE={mae:,.4f} | RMSE={rmse:,.4f} | R2={r2:,.6f}")
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

results = {}

# 2) ElasticNet (log-target), if you saved it
enet_path = ART / "elasticnet_logtarget.joblib"
if enet_path.exists():
    enet = joblib.load(enet_path)
    y_hat_log = enet.predict(X_va)
    y_hat = np.expm1(y_hat_log)
    results["ElasticNet_logtarget"] = report(y_va, y_hat, "ElasticNet-logtarget")

# 3) CatBoost — plain (tuned)
cb_plain_path = ART / "catboost_plain_tuned.cbm"
if cb_plain_path.exists():
    cb_plain = CatBoostRegressor()
    cb_plain.load_model(cb_plain_path)
    y_hat_plain = cb_plain.predict(X_va)
    results["CatBoost_plain_tuned"] = report(y_va, y_hat_plain, "CatBoost-plain (tuned)")

# 4) CatBoost — log (tuned)
cb_log_path = ART / "catboost_log_tuned.cbm"
if cb_log_path.exists():
    cb_log = CatBoostRegressor()
    cb_log.load_model(cb_log_path)
    y_hat_log = cb_log.predict(X_va)
    y_hat = np.expm1(y_hat_log)
    results["CatBoost_log_tuned"] = report(y_va, y_hat, "CatBoost-log (tuned)")

print("\n=== Summary on SAVED VALID ===")
print(json.dumps(results, indent=2))


VALID shapes: (212984, 23) (212984,)
[ElasticNet-logtarget]  MAE=3.6882 | RMSE=59.5958 | R2=-12.965878
[CatBoost-plain (tuned)]  MAE=1.3440 | RMSE=2.7259 | R2=0.970782
[CatBoost-log (tuned)]  MAE=1.2747 | RMSE=2.7156 | R2=0.971002

=== Summary on SAVED VALID ===
{
  "ElasticNet_logtarget": {
    "MAE": 3.688177072188429,
    "RMSE": 59.59578093605666,
    "R2": -12.9658778527889
  },
  "CatBoost_plain_tuned": {
    "MAE": 1.3439523977620846,
    "RMSE": 2.7258696212901383,
    "R2": 0.9707822096565165
  },
  "CatBoost_log_tuned": {
    "MAE": 1.2746886792981627,
    "RMSE": 2.7155846372263657,
    "R2": 0.9710022771084897
  }
}
